In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['M5','M15','H1','H3','D1','W1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    WEEK_DAY = { 'Sunday':1, 'Monday':2, 'Tuesday':3, 'Wednesday':4, 'Thursday':5, 'Friday':6, 'Saturday':7 }

    Src = pd.DataFrame()
    Src['A']        = Query['a']
    Src['Z']        = Query['z']
    Src['Symbol']   = Query['symbol']
    Src['TF']       = Query['tf']
    
    Src['Datetime'] = Query['datetime']
    Src['Date']     = Query['datetime'].dt.date
    Src['Time']     = Query['datetime'].dt.time
    Src['Week.Day'] = Query['datetime'].dt.day_name().map(WEEK_DAY)
    
    Src['Ticks']    = Query['tick_volume']
    Src['Volume']   = Query['real_volume']
    Src['Price']    = Query['close']
    
    Src['Open']     = Query['open']
    Src['High']     = Query['high']
    Src['Low']      = Query['low']
    Src['Close']    = Query['close']

    Src['Change']   = Src['Close'] - Src['Open']
    Src['HL']       = Src['High']  - Src['Low']

    Src['Chg Abs']  = Src['Change'].abs() 
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [33]:
Src.groupby('TF', sort=0)[['HL','Chg Abs']].apply(lambda S: (S > S.mean()).mean().mul(100).round(0).astype(int))

,HL,Chg Abs
TF,,
M5,38,36
M15,39,36
H1,39,36
H3,42,37
D1,42,40
W1,42,40


In [ ]:
# def CALCULATIONS(Src):
#     Calc = pd.DataFrame(Src)
#     Calc['Day Open']    = Calc.groupby(['Symbol','TF','Date'], sort=0)['Open'].transform('first')
#     Calc['Day Close']   = Calc.groupby(['Symbol','TF','Date'], sort=0)['Close'].transform('last')
#     Calc['ID Chg Bwd']  =                     Calc['Open'] - Calc['Day Open']
#     Calc['ID Chg Fwd']  = Calc['Day Close'] - Calc['Open']

#     Calc['ID Chg Fwd Abs'] = Calc['ID Chg Fwd'].abs()
#     Calc['ID Chg Fwd Pos'] = Calc['ID Chg Fwd'].apply(lambda x: x if (x >= 0) else nan)
#     Calc['ID Chg Fwd Neg'] = Calc['ID Chg Fwd'].apply(lambda x: x if (x <  0) else nan)
#     return Calc

# Calc = CALCULATIONS(Src)
# Calc.head()

In [ ]:
# def SIGNALS(Calc):
#     Sign = pd.DataFrame(Calc)
#     for b in [100, 200, 500, 800, 1000, 1200, 1500, 1800, 2000, 2200, 2500, 2800, 3000, 3200, 3500, 3800, 4000, 4200, 4500, 4800, 5000, 5200, 5500, 5800, 6000]:
#         Sign[f'ID Chg Bwd > {b}'] = (Calc['ID Chg Bwd'] > b).astype(int)
#     return Sign

# Sign = SIGNALS(Calc)
# Sign.head()

# Stats

In [ ]:
# def CALCULATIONS(Src):
#     def _int(x):
#         try:    return int(x)
#         except: return x
        
#     Calc = pd.DataFrame(Src)
#     Calc['Day Open']        = Calc.groupby(['Symbol','TF','Date'], sort=0)['Open']  .transform('first')
#     Calc['Day High']        = Calc.groupby(['Symbol','TF','Date'], sort=0)['High']  .transform('max')
#     Calc['Day Low']         = Calc.groupby(['Symbol','TF','Date'], sort=0)['Low']   .transform('min')
#     Calc['Day Close']       = Calc.groupby(['Symbol','TF','Date'], sort=0)['Close'] .transform('last')
#     Calc['Day Chg']         = Calc['Day Close'] - Calc['Day Open']
#     Calc['Day Chg Abs']     = Calc['Day Chg'].abs()

#     Calc['Intra High Rank'] = Calc.groupby(['Symbol','TF','Date'], sort=0)['High'].rank(method='dense', ascending=0)
#     Calc['Intra Low Rank']  = Calc.groupby(['Symbol','TF','Date'], sort=0)['Low'].rank(method='dense', ascending=1)

#     WINS = [0, 100, 200, 500, 800, 1000, 1200, 1500, 1800, 2000, 2200, 2500, 2800, 3000, 3200, 3500, 3800, 4000, 4200, 4500, 4800, 5000, inf]
#     for (a,b) in [x for x in pd.Series(WINS).rolling(2) if (len(x) == 2)]:
#         Calc[f'Day Chg Abs {_int(a)}-{_int(b)}'] = ((a <= Calc['Day Chg Abs']) & (Calc['Day Chg Abs'] < b)) .astype(int)

#     return Calc

# Calc = CALCULATIONS(Src)
# Calc.head(2)

In [ ]:
# def STATS(Calc, bins):
#     pipe = []
#     for x in bins: 
#         Df1 = Calc[Calc[x]==1] #.reset_index(drop=1)

#         for y in ['High','Low']:
#             Df2 = Df1[Df1[f'Intra {y} Rank']==1] 

#             pipe.append({ 'Bins':x, 'HL':y,
#                 'Count':    Df2['Time'].count(),
#                 'Time Avg': np.round(Df2['Time'].apply(lambda x: x.hour + x.minute/60).mean(), 1),
#                 'Time Max': np.round(Df2['Time'].apply(lambda x: x.hour + x.minute/60).max(), 1),
#                 'Time Q3':  np.round(Df2['Time'].apply(lambda x: x.hour + x.minute/60).quantile(.75), 1),
#                 'Time Med': np.round(Df2['Time'].apply(lambda x: x.hour + x.minute/60).median(), 1),
#                 'Time Q1':  np.round(Df2['Time'].apply(lambda x: x.hour + x.minute/60).quantile(.25), 1),
#                 'Time Min': np.round(Df2['Time'].apply(lambda x: x.hour + x.minute/60).min(), 1),
#             })
#         pass
#     pass
#     return pd.DataFrame(pipe)


# Sts = STATS(Calc, bins=Calc.loc[:, f'Day Chg Abs 0-100':f'Day Chg Abs 5000-inf'].columns)
# Sts